In [1]:
!pip install -q sacrebleu sacremoses
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr
import sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 49.4 MB/s eta 0:00:00


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
models = {
    "Hindi": "Helsinki-NLP/opus-mt-en-hi",
    "French": "Helsinki-NLP/opus-mt-en-fr",
    "Spanish": "Helsinki-NLP/opus-mt-en-es"
}

In [4]:
tokenizers = {}
translation_models = {}

In [5]:
for lang, model_name in models.items():
    print(f"Loading {lang} model...")
    tokenizers[lang] = AutoTokenizer.from_pretrained(model_name)
    translation_models[lang] = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

Loading Hindi model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/258 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [6]:
def translate(text, target_language, reference=None):
    tokenizer = tokenizers[target_language]
    model = translation_models[target_language]

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    translated_tokens = model.generate(**inputs, max_length=256)
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    if reference and reference.strip():
        bleu = sacrebleu.sentence_bleu(translated_text, [reference])
        return f"{translated_text}\n\nBLEU Score: {bleu.score:.2f}"

    return translated_text

In [7]:
interface = gr.Interface(    fn=translate,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter text in English...", label="Input Text"),
        gr.Dropdown(choices=list(models.keys()), label="Target Language"),
        gr.Textbox(lines=2, placeholder="Enter reference translation (optional for BLEU)...", label="Reference Translation (Optional)")
    ],
    outputs="text",
    title="Multi-language Translator",
    description="Translate English text to Hindi, French, or Spanish using MarianMT models. Optionally provide a reference to see BLEU score."
)

In [8]:
interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://105a831235fb3bc87f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
